# Databases with SQL

In this workbook we will look at creating and manipulating databases. First I create a datebase in SQLite. SQLite creates portable SQL databases saved in a single file locally, rather than on a remote server. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines. For a production ready application we would use something a bit more heavy duty like PostgreSQL, but to start us off SQLite is ideal.

In [1]:
import numpy as np
from numpy import genfromtxt
import pandas as pd
from pandas.io import sql
import sqlite3

First we create the database.

In [2]:
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

Now we input the database schema.

In [3]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# we save the database by committing it
conn.commit()

Since we're using Python, we can now use regular programming techniques in conjunction with the sqlite connection. In particular, the cursor's execute() method supports value substitution using the ? character, which makes adding multiple records a bit easier.

In [4]:
last_sale = (None, 4000, 5, 22, 619000)

c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', last_sale)

conn.commit()

Notice that in this syntax we use the python *None* value, rather than *NULL* to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method executemany() which takes an array of tuples and loops through them, substituting one tuple at a time.

In [5]:
recent_sales = [(None, 2390, 4, 34, 319000),
                (None, 1870, 3, 14, 289000),
                (None, 1505, 3, 90, 269000),]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Next let's load our housing.csv data into an array, and then INSERT those records into the database. In this example we'll use the numpy genfromtxt function to read the file and parse the contents.

In [6]:
# import into Numpy array of integers, then convert to list of lists
data = (genfromtxt('data/housing.csv', dtype='i8', delimiter=',', skip_header = 1)).tolist()

In [7]:
# we can append a None value to the beginning of each list

for d in data:
    d.insert(0, None)

Why did we do this? Well, all elements in a numpy array must be the same data type, so if we want to 'add a *None*' to each row, we need to work around this. Note that lists, however, can contain mixed data types.

Still, in this case the value we're adding is the same for all records, so we could have simply used a 'None' in the INSERT statement directly.

In [8]:
# loop through data, running an INSERT on each record ( or sublist)
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [9]:
# we can make a query as follows
results = c.execute('SELECT * FROM houses WHERE bdrms = 2')

# here 'results' is a cursor object - we use fetchall() to extract a list
results.fetchall()

[(8, 1416, 2, 49, 232000),
 (21, 1320, 2, 62, 299900),
 (26, 1888, 2, 79, 255000),
 (35, 1839, 2, 40, 349900),
 (45, 1664, 2, 40, 368500),
 (49, 852, 2, 70, 179900)]

## Connecting with Pandas

Databases provide many analytical capabilities, such that large, fixed operations would be more efficient when performed in this way. For example, if you want to aggregate nightly log-ins or sales to present a report or dashboard, then this operation is likely not changing that much, if at all. This can run very efficiently in a database rather than by connecting to it with Python.

However, if we want to investigate login or sales data further and ask more interactive questions, then Python would be more practical. It's often useful to pull the data back into Python (and use Pandas) as it's more flexible and has interactive programming operations.

In [10]:
data = pd.read_csv('data/housing.csv')
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


In [11]:
data.to_sql('db_pandas',
            con = conn,
            if_exists = 'replace',
            index = False)

In [12]:
# the core query is in the same format as above
sql.read_sql('SELECT * FROM db_pandas LIMIT 5', con = conn)

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


## SQL Queries

Following are a range of queries in SQL. The syntax is straightforward and is generally easy to interpret.

In [13]:
sql.read_sql('SELECT sqft, bdrms FROM db_pandas LIMIT 5', con = conn)

,sqft,bdrms
0,2104,3
1,1600,3
2,2400,3
3,1416,2
4,3000,4


In [14]:
sql.read_sql('SELECT sqft, bdrms FROM db_pandas WHERE sqft < 2000 AND bdrms = 2 LIMIT 5', con = conn)

,sqft,bdrms
0,1416,2
1,1320,2
2,1888,2
3,1839,2
4,1664,2


In [15]:
sql.read_sql('SELECT COUNT (price) FROM db_pandas', con = conn)

,COUNT (price)
0,47


In [16]:
sql.read_sql('SELECT AVG (sqft), MIN(price), MAX(price) FROM db_pandas WHERE bdrms = 2', con = conn)

,AVG (sqft),MIN(price),MAX(price)
0,1496.5,179900,368500


In [17]:
sql.read_sql('SELECT AVG (price) FROM db_pandas WHERE bdrms = 1', con = conn)

,AVG (price)
0,169900.0


In [18]:
sql.read_sql('SELECT AVG (price) FROM db_pandas WHERE bdrms = 2', con = conn)

,AVG (price)
0,280866.666667


In [19]:
sql.read_sql('SELECT COUNT (bdrms) FROM db_pandas GROUP BY bdrms', con = conn)

,COUNT (bdrms)
0,1
1,6
2,25
3,14
4,1


In [20]:
sql.read_sql('SELECT COUNT (bdrms) FROM db_pandas WHERE bdrms = 3', con = conn) / sql.read_sql('SELECT COUNT (bdrms) FROM db_pandas', con = conn)

,COUNT (bdrms)
0,0.531915


In [21]:
sql.read_sql('SELECT MAX(age) FROM db_pandas GROUP BY bdrms', con = conn)

,MAX(age)
0,5
1,79
2,78
3,77
4,49


In [22]:
sql.read_sql('SELECT MIN(age) FROM db_pandas', con = conn)

,MIN(age)
0,5


In [23]:
sql.read_sql('SELECT AVG(age) FROM db_pandas', con = conn)

,AVG(age)
0,42.744681


In [24]:
sql.read_sql('SELECT AVG(age) FROM db_pandas GROUP BY bdrms', con = conn)

,AVG(age)
0,5.000000
1,56.666667
2,38.360000
3,46.857143
4,49.000000
